# Testing on PISA dataset with randomized missing data

1. Importing package and dataset

In [8]:
!pip install iita_python
!git clone https://gist.github.com/717f0147675b0c8ed25e50d583c943bf.git

import numpy as np
import iita_python as iita
import iita_python.fit_metrics as iita_fm
from iita_python.utils import read_rp
from random import randint

data = read_rp('./717f0147675b0c8ed25e50d583c943bf/pisa.csv')

fatal: destination path '717f0147675b0c8ed25e50d583c943bf' already exists and is not an empty directory.


2. Testing function

In [21]:
def test(metric):
  correct = True
  correct_qo = None
  correct_count = 0

  while (correct and correct_count < data.shape[0]*data.shape[1]):
    print(f'doing {correct_count}')
    test_dataset = iita.Dataset(data)
    unfolded_ce = iita.unfold_examples(test_dataset.ce)
    qos = iita.ind_gen(unfolded_ce, test_dataset.items)

    best_qo_id = -1
    best_qo_diff = float('inf')
    for i, qo in enumerate(qos):
      qo_diff = metric(test_dataset, qo)
      if (qo_diff < best_qo_diff):
        best_qo_diff = qo_diff
        best_qo_id = i

    best_qo = sorted([(int(a), int(b)) for a, b in qos[best_qo_id].get_edge_list()])
    if (correct_qo is None):
      correct_count += 1
      correct_qo = best_qo
    elif (best_qo != correct_qo):
      correct = False
    else:
      correct_count += 1

      while (True):
        a = randint(0, test_dataset.items - 1)
        b = randint(0, test_dataset.subjects - 1)
        if (not np.isnan(data[a][b])):
          break;
      data.loc[b, a] = np.nan

  return correct_count

In [ ]:
res = test(iita_fm.orig_iita_fit)

In [23]:
res

525